In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

training_data = pd.read_csv('data/mnist_train.csv')

/Users/faridjamshid/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
training_data = np.array(training_data)
m, n = training_data.shape

Y_train = training_data[:,0]
X_train = training_data[:, 1:].T

X_train = X_train / 255

In [3]:
def ReLU(Z):
    return np.maximum(Z, 0)

def init_params():
    W1 = np.random.randn(64, 784) * np.sqrt(2. / 784)
    b1 = np.zeros((64, 1))
    W2 = np.random.randn(10, 64) * np.sqrt(2. / 64)
    b2 = np.zeros((10, 1))
    
    return W1, b1, W2, b2

def ReLU_deriv(Z):
    return Z > 0

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True)) 
    return expZ / np.sum(expZ, axis=0, keepdims=True)

def forw_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def back_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    return W1, b1, W2, b2

In [4]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forw_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print(f'iteration: {i}')
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [5]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.1, 2001)

iteration: 0
[2 2 3 ... 4 7 2] [5 0 4 ... 5 6 8]
0.09806666666666666
iteration: 10
[0 0 4 ... 3 6 8] [5 0 4 ... 5 6 8]
0.6159
iteration: 20
[5 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.7478666666666667
iteration: 30
[5 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8008666666666666
iteration: 40
[5 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8285833333333333
iteration: 50
[5 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8448666666666667
iteration: 60
[5 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8555166666666667
iteration: 70
[5 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.86355
iteration: 80
[5 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8699666666666667
iteration: 90
[5 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8748
iteration: 100
[5 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8787
iteration: 110
[5 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8821166666666667
iteration: 120
[5 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8848833333333334
iteration: 130
[5 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.8878166666666667
iteration: 140
[5 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.89
iteration: 150
[5 0 4

In [6]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forw_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

In [7]:
test_data = pd.read_csv('data/mnist_train.csv')
test_data = np.array(test_data)

Y_test = test_data[:, 0]
X_test = test_data[:, 1:].T

In [8]:
predictions = make_predictions(X_test, W1, b1, W2, b2)
print(get_accuracy(predictions, Y_test))

[5 0 4 ... 5 6 8] [5 0 4 ... 5 6 8]
0.9491833333333334


In [9]:
import pickle

with open("mnist_weights.pkl", "wb") as f:
    pickle.dump((W1, b1, W2, b2), f)